# 분류 모델 성능 평가 
= 예측 대상이 범주형
- 정확도(Accuracy)
- 재현율(Recall)
- 정밀도(Precision)
- F1 measure
- G measure
- ROC curve
- AUC

# 회귀 모델 평가 지표
= 예측 대상이 수치 데이터
- MSE(Mean Square Error)
- RMSE(Root Mean Square Error)
- MAE(Mean Absolute Error)
- MAPE(Mean Absolute Percentage Error)
- R^2

In [1]:
import numpy as np
from sklearn.base import BaseEstimator

# 간단한 분류기 만들기
class MyDummyClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass

    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        return pred

In [8]:
from sklearn.preprocessing import LabelEncoder

# Null 처리
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Fare'].fillna(0,inplace=True)
    df.fillna('N', inplace=True)
    return df

# 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis =1, inplace=True)
    return df

# 레이블 인코딩
def format_features(df):
    df['Cabin'] = df['Cabin'].str[0]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

titanic_df = pd.read_csv('./data/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train,X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                  test_size=0.2, random_state=0)

In [10]:
# 간단한 분류기로 확인

myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)
mypred = myclf.predict(X_test)
print('Dummy Classifier의 정확도 : {0:.4f}'.format(accuracy_score(y_test, mypred)))

Dummy Classifier의 정확도 : 0.7877


## MNIST 데이터 세트
- 0~9까지 숫자 이미지의 픽셀 정보를 가지고 이를 기반으로 숫자 Digit를 예측하는 데 사용
- sklearn load_digits()

**이진 분류문제로 변환**
- 불균형한 데이터 세트로 변형
- 레이블 값이 7인 것만 true, 나머지 false
- true 전체 데이터 10%

### 정확도 평가 지표의 맹점
- 아무것도 안하고 랜덤하게 결과를 찍어도 데이터가 균일하지 않은 경우 높은 예측치가 나타날 수 있음

In [21]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    def predict(self, X):
        return np.zeros((len(X),1), dtype = bool)


In [12]:
digits = load_digits()
digits

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [14]:
digits.data.shape

(1797, 64)

In [15]:
digits.target.shape

(1797,)

**7인 데이터 확인**

In [16]:
digits.target == 7

array([False, False, False, ..., False, False, False])

In [17]:
y = (digits.target==7).astype(int)

**학습/테스트 데이터 분리**

In [18]:
X_train, X_test, y_train, y_test = \
train_test_split(digits.data, y, random_state=11) # test_size 기본값 0.25

**불균형한 레이블 데이터 분포도 확인**

In [19]:
pd.Series(y_test).value_counts()

0    405
1     45
dtype: int64

In [22]:
fake_cl = MyFakeClassifier()

fake_cl.fit(X_train, y_train)
fakepred = fake_cl.predict(X_test)
accuracy = accuracy_score(y_test, fakepred)

print('정확도 : ', np.round(accuracy,3))

정확도 :  0.9


## Confusion Matrix(오차행렬)
**오차행렬**
- 이진 분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타내는 지표
- 예측 클래스와 실제 클래스의 값 유형에 따라 TN,FP,FN,TP형태



In [23]:
from sklearn.metrics import confusion_matrix
# MNIST의 fakepred
confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]], dtype=int64)

## 정밀도와 재현율
- 정밀도 = TP/(FP + TP)
- 재현율 = TP/(FN + TP)